-----
# cartoGRAPHs - FIGURE 2A
precalculated large networks , e.g. the Interactome

A Notebook to produce 2D and 3D network layouts from any Graph,
including interactive visualization (html files) and export functions 
to import into the VRNetzer analytics platform by Pirch et al.

-----

In [1]:
from cartoGRAPHs import * 

from func_load_data import *
from func_visual_properties import * 
from func_calculations import * 
from func_embed_plot import * 
from func_exportVR import * 

_____
# HUMAN NETWORK
_____

In [2]:
organism = 'human'

G = load_graph(organism) 

d_gene_sym = load_genesymbols(G, organism)
l_features = list(d_gene_sym.values())

d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
d_names_do = {y:x for x,y in d_do_names.items()}

In [3]:
d_centralities = load_centralities(G, organism)
df_centralities = pd.DataFrame(d_centralities).T
df_centralities.columns = ['degree','closeness', 'betweeness', 'eigenvector']
essential_genes,non_ess_genes,notdefined_genes = load_essentiality(G, organism)

_____ 
# 2 | FIGURE 2A + supplementary | Importance layout (with diverse node lists highlighted)
_____

In [4]:
DM_cent = load_datamatrix(G,organism, 'importance')
DM_cent_mod = DM_cent.round(5)

### 2D PORTRAIT

In [199]:
n_n = 20
spr = 1.0
md = 0.0
metr = 'cosine'

In [ ]:
%%time 

posG_umap2D = layout_portrait_umap(G,DM_cent_mod, 2,
                                   n_neighbors = n_n,
                                   spread = spr,
                                   min_dist = md,
                                   metric = metr) 
posG = posG_umap2D

## VISUAL SETTINGS + PLOT PREPARATION

#### NODES - GENERAL

In [201]:
opacity_nodes = 0.8
node_edge_col = '#696969' 
node_linewidth = 1

scale_factor = 0.5
#size_nx = list(draw_node_degree(G, scale_factor).values())
size_plotly = list(draw_node_degree(G, scale_factor/100).values())

scale_factor3D = 0.025
size3d = list(draw_node_degree_3D(G, scale_factor3D).values())

nodesglow_diameter = 8.0
nodesglow_transparency = 0.01 # 0.01

#### ESSENTIALITY color settings 

In [ ]:
color_method = 'essentiality'

l_genes = essential_genes
    
node_col = '#00abff' 
undefined_col = '#DEDEDE '
rest_col_nodes = '#DEDEDE '

d_col_all = color_nodes_from_list(G, essential_genes, node_col)
colors = list(d_col_all.values())
    
edge_color = node_col 
d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)
width_edges = 0.25
opacity_edges = 0.25

#### DEVELOPMENTAL color settings 

In [ ]:
color_method = 'oocyte'

if color_method == 'oocyte' or color_method == '2cell' or color_method == '4cell' or color_method == '8cell':

    node_col = '#0181A3' 
    devgenes_selection = color_method
    df_devgenes = pd.read_csv('input/EmExplorer/'+devgenes_selection+'_Homo-sapiens.txt', delimiter = "\t")
    
    devgenes = list(df_devgenes['stage'])

    d_devgenes_entrezid = {}
    for i,k in d_gene_sym.items():
        if k in devgenes:
             d_devgenes_entrezid[k] = i 
    devgenes_entrezid = list(d_devgenes_entrezid.values())
    
    l_genes = devgenes_entrezid
    
    d_col_all = color_nodes_from_list(G, l_genes, node_col)
    colors = list(d_col_all.values())

    edge_color = node_col
    d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)
    width_edges = 0.25
    opacity_edges = 0.5
        
else:
    pass

#### CANCER color settings 

In [181]:
disease_category = 'cancer'

color_method = disease_category
disease_genes = get_disease_genes(G, d_names_do, d_do_genes, disease_category)
l_genes = disease_genes

node_col = '#00C9FF' 
d_col_all = color_nodes_from_list(G, disease_genes, node_col)
colors = list(d_col_all.values())

edge_color = node_col 
d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)

width_edges = 0.25
opacity_edges = 0.25

#### RARE DISEASE genes color settings 

In [76]:
import xml.etree.ElementTree as ET
tree = ET.parse('input/orphadata.xml')
root = tree.getroot()

# retrieve genes from all rare diseases from database/file  
disorder_to_genes = {}
for disorder in root.iter('Disorder'):
    dis = list(disorder.attrib.values())[0]
    for elem in disorder.findall('DisorderGeneAssociationList'):
        genes = []
        for disorderassoc in elem.findall('DisorderGeneAssociation'):
            for gene in disorderassoc.findall('Gene'):
                for reference in gene.findall('ExternalReferenceList'):
                     for refs in reference.findall('ExternalReference'):
                        for ensg in refs.findall('Source'):
                            if ensg.text == 'Ensembl':
                                for e in refs.findall('Reference'):
                                    ensgID = (e.text)
                            else:
                                pass
                            
            genes.append(ensgID)
    disorder_to_genes[dis] = genes

    
# get a list of all rare disease genes obtained 
all_raredisease_genes = []
for dis,genes in disorder_to_genes.items():
    for i in genes:
        all_raredisease_genes.append(i)
      
    
# make sure there are no duplicates
set_all_raredisease_genes = set(all_raredisease_genes)


# convert ensgID to entrezID and get gene symbols
ensg_to_entrez = pd.read_csv('input/ensg_to_entrezid.txt',sep="	")

ensg = list(ensg_to_entrez['From'])
entrez = [str(int(i)) for i in ensg_to_entrez['To'][:-1]] #list(ensg_to_entrez['To'])
d_ensg_entrez = dict(zip(ensg,entrez))


# match ensgID with entrezID
set_all_raredisease_genes_id = {}
for ix,entrez in d_ensg_entrez.items():
    if ix in set_all_raredisease_genes:
        set_all_raredisease_genes_id[ix]=entrez

        
# color all rare disease genes  
l_rarediseasegenes = list(set_all_raredisease_genes_id.values())
color_method = 'rarediseasegenes'

l_genes = l_rarediseasegenes

node_col =  '#004EAB'#'#0A81A1' 
undefined_col = '#DEDEDE '
rest_col_nodes = '#DEDEDE '

d_col_all = color_nodes_from_list(G, l_genes, node_col)
colors = list(d_col_all.values())
    
edge_color = node_col 
d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)
width_edges = 0.25
opacity_edges = 0.25

## PLOT NETWORK PORTRAIT - plotly 

In [77]:
# plot nodes based on focus > separate to background/foreground 

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree(G, scale_factor/100)
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())

In [81]:
umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, 
                                           node_linewidth)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, 
                                           node_linewidth*0.25)
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 0.5, node_linewidth*0.5)

umap_edges = get_trace_edges_specific2D(d_edge_col, posG, linew=width_edges, opac=opacity_edges)
data = [umap_edges, 
    umap_nodes_background_, umap_nodes_background, umap_nodes_foreground]

path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
fname = '2Dportrait_NetlayoutImportance_'+color_method+'_'+organism

plot_2D(data,path,fname)

'output_plots/2Dlayouts/2Dportrait_NetlayoutImportance_rarediseasegenes_human_onlynodes.html'

## PLOT NETWORK PORTRAIT - networkx

In [ ]:
# Choose edges to highlight that link inbetween 2 essential genes only 
d_edge_col = color_edges_from_nodelist_specific(G, essential_genes, node_col)

# Rest edges 
d_rest_edges={}
for e in G.edges():
    if e not in d_edge_col.keys():
        d_rest_edges[e] = '#f0f0f0'

d_all_edges = {**d_edge_col, **d_rest_edges}
d_all_edges_sort = {key:d_all_edges[key] for key in G.edges()}
edge_color = list(d_all_edges_sort.values())



plt.figure(figsize=(25,25))
plt.title('2D Portrait', size=16)

nx.draw_networkx_nodes(G, 
                       posG,
                       edgecolors = 'dimgrey', 
                       linewidths = 1.2, 
                       node_color = colors, 
                       node_size = size_nx, 
                       alpha = 0.8)

nx.draw_networkx_edges(G, posG, width = 0.1, 
                       edge_color = edge_color, 
                      alpha = 0.25)

plt.box(False)

fname = 'output_plots/2Dlayouts/2Dportrait_Fig2a_'+organism

# PNG
#plt.savefig(fname+'.png',dpi=300)

# PDF 
plt.savefig(fname+'.pdf',format='pdf')

plt.show()